## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 4% des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d erreurs résiduelles](#8---Exemples-d-erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [68]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

In [69]:
def analyse_integrite(data, fields, pr=True):
    '''analyse les relations du DataFrame définies dans le dict fields et retourne un dict avec pour chaque contrôle la
    liste des index ko. Les résultats des contrôles sont également ajoutés sous forme de champs booléens à data'''
    analyse = Analysis(data)
    dic_res = analyse.check_relationship(fields)
    data['ok'] = True
    for name, lis in dic_res.items():
        data[name] = True
        data.loc[lis, name] = False
        data['ok'] = data['ok'] & data[name]
        if pr:
            print('{:<50} {:>5}'.format(name, len(data) - data[name].sum()))
    return dic_res

------
# 2 - Initialisation
## initialisation des données
- lecture du fichier issu de l'api

In [241]:
from datetime import datetime
import json
from observation import Ilist, Analysis
import pandas as pd
#date = '2023-03-03'
date = '2023-04-17'
log = {'date_irve': date, 
       'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
      'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/Analyse/'}
#log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
#      'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\'}
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  66323


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [242]:
fields = [
 
 # relation unicité des pdl
 { "name": "index",
   "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory =  ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
           'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
           'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
           'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
           'date_maj', 'last_modified']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- un peu moins de 1 % des points de charge sont hors itinerance

In [243]:
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'])
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  650
nombre de pdc en itinerance   :  65673


In [244]:
# arborescence des champs et nombre de valeurs différentes
il = Ilist.obj(itinerance_init)
print(il.tree())

-1: root-derived (65673)
   0 : index (65673)
      1 : contact_operateur (124)
      2 : nom_enseigne (1141)
      3 : coordonneesXY (17261)
      4 : adresse_station (16523)
      5 : id_station_itinerance (18949)
      6 : nom_station (15493)
      7 : implantation_station (5)
      8 : nbre_pdc (52)
      9 : condition_acces (2)
      10: horaires (510)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (46161)
      13: date_maj (578)
      14: last_modified (494)


-----------------------
## 4 - Bilan initial intégrité
- 36 % des lignes sont erronées ( 18 116 )

In [245]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          29007
contact_operateur - id_station_itinerance          12919
nom_enseigne - id_station_itinerance                9507
coordonneesXY - id_station_itinerance              14069
id_station_itinerance - id_pdc_itinerance            564
nom_station - id_station_itinerance                 2069
implantation_station - id_station_itinerance        1296
nbre_pdc - id_station_itinerance                    4756
condition_acces - id_station_itinerance               31
horaires - id_station_itinerance                   12175
station_deux_roues - id_station_itinerance         13742
adresse_station - coordonneesXY                     1437

nombre d'enregistrements sans erreurs :  33612
nombre d'enregistrements avec au moins une erreur :  32061
taux d'erreur :  49  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- la moitié des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de diviser par 5 le nombre de lignes erronnées ( 2 166 )

In [246]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'])
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'])

nombre de doublons pdc :  19512
nombre de pdc sans doublon   :  12549


In [247]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance            270
nom_enseigne - id_station_itinerance                 455
coordonneesXY - id_station_itinerance               1058
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  535
implantation_station - id_station_itinerance         227
nbre_pdc - id_station_itinerance                     940
condition_acces - id_station_itinerance               13
horaires - id_station_itinerance                     296
station_deux_roues - id_station_itinerance           716
adresse_station - coordonneesXY                      631

nombre d'enregistrements sans erreurs :  9556
nombre d'enregistrements avec au moins une erreur :  2993


-----------------------
## 6 - Séparation doublons station - date de maj
- 20% des erreurs résiduelles sont liées au mélange d'anciens et de nouveaux pdc
- la suppression des anciens pdc permet de réduire de 25% le nombre de lignes erronnées ( 1 616 )
- les dernières erreurs correspondent à 468 stations associées à 32 opérateurs et sont liées à des causes multiples

In [248]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'])
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  433
nombre de pdc sans doublon   :  2560


In [249]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                 267
coordonneesXY - id_station_itinerance                537
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  339
implantation_station - id_station_itinerance          88
nbre_pdc - id_station_itinerance                     842
condition_acces - id_station_itinerance                8
horaires - id_station_itinerance                      81
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                      414

nombre d'enregistrements sans erreurs :  225
nombre d'enregistrements avec au moins une erreur :  2335


In [250]:
# structure des données résiduelles et nombre de valeurs
il = Ilist.obj(itinerance_5)
print(il.tree())

-1: root-derived (2335)
   0 : index (2335)
      1 : contact_operateur (32)
      2 : nom_enseigne (325)
      3 : coordonneesXY (587)
      4 : adresse_station (629)
      5 : id_station_itinerance (561)
         13: date_maj (81)
      6 : nom_station (599)
      7 : implantation_station (5)
      8 : nbre_pdc (21)
      9 : condition_acces (2)
      10: horaires (93)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (2335)
      14: last_modified (57)


In [251]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  55


-----------------------
## 7 - Synthèse


### fichiers

In [252]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  2335
total des doublons à supprimer :  19945
nombre de pdc avec controles ok :  43393


In [253]:
#fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
#fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
#fichier csv des données itinerance valides (IRVE_itinerance_valide)
#fichier csv des doublons (IRVE_itinerance_doublons)
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [254]:
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, pr=False)
log['bilan_erreurs'] = sum([len(controle) for controle in res.values()])
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [255]:
# structure des données bonnes
il = Ilist.obj(itinerance_ok.loc[:, mandatory])
print(il.tree())

-1: root-derived (43393)
   11: id_pdc_itinerance (43393)
      4 : id_station_itinerance (18326)
         1 : nom_enseigne (821)
         2 : coordonneesXY (14432)
            0 : contact_operateur (107)
            3 : adresse_station (13834)
               8 : condition_acces (2)
         5 : nom_station (14484)
         6 : implantation_station (5)
         7 : nbre_pdc (52)
         9 : horaires (425)
         10: station_deux_roues (5)
         19: paiement_autre (7)
      12: puissance_nominale (81)
      13: prise_type_ef (8)
      14: prise_type_2 (8)
      15: prise_type_combo_ccs (7)
      16: prise_type_chademo (7)
      17: prise_type_autre (7)
      18: paiement_acte (8)
      20: reservation (6)
      21: accessibilite_pmr (4)
      22: restriction_gabarit (96)
      23: date_maj (432)
      24: last_modified (367)


### Indicateurs

In [256]:
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)

log |= indic

In [257]:
with open('logfile.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')
print(log)

{'date_irve': '2023-04-17', 'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-20230417.csv', 'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/Analyse/', 'len_irve': 66323, 'pdc_hors_itinerance': 650, 'pdc_en_itinerance': 65673, 'init_ok': 33612, 'init_ko': 32061, 'doublons_pdc': 19512, 'sans_doublons_pdc': 12549, 'etape3_ok': 9556, 'etape3_ko': 2993, 'doublons_station': 433, 'sans_doublons_station': 2560, 'etape5_ok': 225, 'etape5_ko': 2335, 'IRVE_itinerance_residuel2023-04-17.csv': 2335, 'IRVE_itinerance_complet2023-04-17.csv': 65673, 'IRVE_itinerance_valide2023-04-17.csv': 43393, 'IRVE_itinerance_valide_stat2023-04-17.csv': 18326, 'IRVE_itinerance_doublons2023-04-17.csv': 19945, 'bilan_erreurs': 0, 'date': '2023-04-26T23:58:36.413152', 'Pdc non unique': 0, 'Station multi-operateurs': 2, 'Station multi-enseignes': 267, 'Station multi-localisations': 537, 'Pdc multi-stations': 0, 'station avec plusieurs noms': 339, 's

-----------------------
## 8 - Exemples d erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (2 stations soit 12 pdc avec : adresse, nom, nbre_pdc et deux-roues erronés)

In [258]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2267,65913,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",AVENUE DE PARIS VERSAILLES,FR55CP78000,AVENUE DE PARIS VERSAILLES,Voirie,4,Accès réservé,24/7,FALSE,FR55CE780004880188021299331,2022-11-03,2023-04-17T09:38:44.078000+00:00
2269,65916,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",AVENUE DE PARIS VERSAILLES,FR55CP78000,AVENUE DE PARIS VERSAILLES,Voirie,4,Accès réservé,24/7,FALSE,FR55CE780004880188021299351,2022-11-03,2023-04-17T09:38:44.078000+00:00
2347,66016,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.577631, 43.279636]",PARKING AIRE DE ST PONS - GRIMAUD,FR55CP83310,PARKING AIRE DE ST PONS - GRIMAUD,Voirie,3,Accès réservé,24/7,FALSE,FR55CE83310GIRVATMX029BAD1,2022-11-03,2023-04-17T09:38:44.078000+00:00
2348,66017,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.577631, 43.279636]",PARKING AIRE DE ST PONS - GRIMAUD,FR55CP83310,PARKING AIRE DE ST PONS - GRIMAUD,Voirie,3,Accès réservé,24/7,FALSE,FR55CE83310GIRVATMX029BAD2,2022-11-03,2023-04-17T09:38:44.078000+00:00
2349,66018,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.577631, 43.279636]",PARKING AIRE DE ST PONS - GRIMAUD,FR55CP83310,PARKING AIRE DE ST PONS - GRIMAUD,Voirie,3,Accès réservé,24/7,FALSE,FR55CE83310GIRVATMX029BAD3,2022-11-03,2023-04-17T09:38:44.078000+00:00
2365,66041,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.57743, 43.274761]",PARKING PORT - GRIMAUD,FR55CP83310,PARKING PORT - GRIMAUD,Voirie,2,Accès réservé,24/7,FALSE,FR55CE83310PLU00072,2022-11-03,2023-04-17T09:38:44.078000+00:00
2369,66049,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.522957, 43.274431]",PARKING MAIRIE - GRIMAUD,FR55CP83310,PARKING MAIRIE - GRIMAUD,Voirie,2,Accès réservé,24/7,FALSE,FR55CE83310PLU00051,2022-11-03,2023-04-17T09:38:44.078000+00:00
2370,66050,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.522957, 43.274431]",PARKING MAIRIE - GRIMAUD,FR55CP83310,PARKING MAIRIE - GRIMAUD,Voirie,2,Accès réservé,24/7,FALSE,FR55CE83310PLU00052,2022-11-03,2023-04-17T09:38:44.078000+00:00
2371,66051,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.57743, 43.274761]",PARKING PORT - GRIMAUD,FR55CP83310,PARKING PORT - GRIMAUD,Voirie,2,Accès réservé,24/7,FALSE,FR55CE83310PLU00071,2022-11-03,2023-04-17T09:38:44.078000+00:00
2372,66052,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.521344, 43.272782]",PARKING RN7 - GRIMAUD,FR55CP83310,PARKING RN7 - GRIMAUD,Voirie,2,Accès réservé,24/7,FALSE,FR55CE83310PLU00081,2022-11-03,2023-04-17T09:38:44.078000+00:00


### Cohérence implantation_station - id_station
- 23 pdc sont liés à une erreur de choix d'implantation (3 stations, 1 opérateur)

In [259]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1505,47760,contact@qovoltis.com,QOVOLTIS,"[1.8088319, 43.6986226]",1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,FRQOVP8100001,1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,Parking public,6,Accès libre,24/7,0,FRQOVE206294,2022-05-10,2023-04-17T09:42:33.587000+00:00
1507,47785,contact@qovoltis.com,QOVOLTIS,"[1.8088319, 43.6986226]",1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,FRQOVP8100001,1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,Parking public,6,Accès libre,24/7,0,FRQOVE205517,2022-05-10,2023-04-17T09:42:33.587000+00:00
1508,47786,contact@qovoltis.com,QOVOLTIS,"[1.8088319, 43.6986226]",1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,FRQOVP8100001,1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,Parking public,6,Accès libre,24/7,0,FRQOVE205909,2022-05-10,2023-04-17T09:42:33.587000+00:00
1509,47787,contact@qovoltis.com,QOVOLTIS,"[1.8088319, 43.6986226]",1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,FRQOVP8100001,1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,Parking public,6,Accès libre,24/7,0,FRQOVE200919,2022-05-10,2023-04-17T09:42:33.587000+00:00
1510,47801,contact@qovoltis.com,QOVOLTIS,"[1.8088319, 43.6986226]",1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,FRQOVP8100001,1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,Parking public,6,Accès libre,24/7,0,FRQOVE201117,2022-05-10,2023-04-17T09:42:33.587000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2547,66305,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.58115, 43.280687]",PRAIRIE DE LA MER GRIMAUD,FR55CP83310,PRAIRIE DE LA MER GRIMAUD,Voirie,4,Accès réservé,24/7,FALSE,FR55CEFR833101910L1TT0RAL2,2022-11-03,2023-04-17T09:38:44.078000+00:00
2548,66306,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.58115, 43.280687]",PRAIRIE DE LA MER GRIMAUD,FR55CP83310,PRAIRIE DE LA MER GRIMAUD,Voirie,4,Accès réservé,24/7,FALSE,FR55CEFR833101910L1TT0RAL3,2022-11-03,2023-04-17T09:38:44.078000+00:00
2549,66307,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.581173, 43.276785]",PRAIRIE DE LA MER ENTREE GRIMAUD,FR55CP83310,PRAIRIE DE LA MER ENTREE GRIMAUD,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR83310335AVDELAMER0,2022-11-03,2023-04-17T09:38:44.078000+00:00
2550,66308,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.581173, 43.276785]",PRAIRIE DE LA MER ENTREE GRIMAUD,FR55CP83310,PRAIRIE DE LA MER ENTREE GRIMAUD,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR83310335AVDELAMER1,2022-11-03,2023-04-17T09:38:44.078000+00:00


### Cohérence nom_station - id_station
- 40 pdc sont associés à une station avec un nom non cohérent

In [260]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
44,7456,service-recharge@soregies.fr,ALTERBASE86,"[0.37178, 46.56869]","78, avenue Jacques Coeur - 86000 - Poitiers",FRS86PSOREB140,Sorégies 3,Parking privé à usage public,2,Accès libre,Mo-Fr 8:00-19:00,false,FRS86ESOREB1402,2020-04-30,2023-04-17T09:44:56.836000+00:00
45,7464,service-recharge@soregies.fr,ALTERBASE86,"[0.37178, 46.56869]","78, avenue Jacques Coeur - 86000 - Poitiers",FRS86PSOREB140,Sorégies 2,Parking privé à usage public,2,Accès libre,Mo-Fr 8:00-19:00,false,FRS86ESOREB1401,2020-04-30,2023-04-17T09:44:56.836000+00:00
508,10691,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM021,2023-04-13,2023-04-17T09:44:53.352000+00:00
509,10692,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM020,2023-04-13,2023-04-17T09:44:53.352000+00:00
510,10693,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM019,2023-04-13,2023-04-17T09:44:53.352000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,66314,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.267834, 48.808019]",AVENUE JEAN JAURES - CLAMART,FR55CP92140,AVENUE JEAN JAURES - CLAMART,Voirie,5,Accès réservé,24/7,FALSE,FR55CEFR92140137JAURES1,2022-11-03,2023-04-17T09:38:44.078000+00:00
2556,66315,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.267834, 48.808019]",AVENUE JEAN JAURES - CLAMART,FR55CP92140,AVENUE JEAN JAURES - CLAMART,Voirie,5,Accès réservé,24/7,FALSE,FR55CEFR92140137JAURES2,2022-11-03,2023-04-17T09:38:44.078000+00:00
2557,66316,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.267834, 48.808019]",AVENUE JEAN JAURES - CLAMART,FR55CP92140,AVENUE JEAN JAURES - CLAMART,Voirie,5,Accès réservé,24/7,FALSE,FR55CEFR92140137JAURES3,2022-11-03,2023-04-17T09:38:44.078000+00:00
2558,66317,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.267834, 48.808019]",AVENUE JEAN JAURES - CLAMART,FR55CP92140,AVENUE JEAN JAURES - CLAMART,Voirie,5,Accès réservé,24/7,FALSE,FR55CEFR92140137JAURES4,2022-11-03,2023-04-17T09:38:44.078000+00:00


### Cohérence adresse - coordonnées
- 414 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [261]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,403,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA PLACE ETIENNE DOLET,FRSIGPSIGE34211,SIGEIF - 4 RUE CRISTINO GARCIA PLACE ETIENNE D...,Voirie,1,Accès libre,24/7,false,FRSIGESIGE34211,2023-04-18,2023-04-18T02:53:28.820000+00:00
1,550,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA,FRSIGPSIGE34411,SIGEIF - 4 RUE CRISTINO GARCIA PLACE ETIENNE D...,Voirie,1,Accès libre,24/7,false,FRSIGESIGE34411,2023-04-18,2023-04-18T02:53:28.820000+00:00
2,2753,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL AÉROVILLE,FRURWPUNIB1712,AEROVILLE - TOKYO,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1712,2023-04-18,2023-04-18T02:53:23.840000+00:00
3,2754,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAÉROVILLE,FRURWPUNIB1312,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1312,2023-04-18,2023-04-18T02:53:23.840000+00:00
4,2756,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL AÉROVILLE,FRURWPUNIB1711,AEROVILLE - TOKYO,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1711,2023-04-18,2023-04-18T02:53:23.840000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2434,66130,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",6 AVENUE DE PARIS -VERSAILLES,FR55CP78000,6 AVENUE DE PARIS -VERSAILLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CEFR78000AVDEPAR1S1,2022-11-03,2023-04-17T09:38:44.078000+00:00
2438,66142,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",6 AVENUE DE PARIS -VERSAILLES,FR55CP78000,6 AVENUE DE PARIS -VERSAILLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CEFR78000AVDEPAR1S0,2022-11-03,2023-04-17T09:38:44.078000+00:00
2541,66299,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP731,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S0,2022-11-03,2023-04-17T09:38:44.078000+00:00
2552,66310,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP731,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S1,2022-11-03,2023-04-17T09:38:44.078000+00:00


- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [262]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[0.193942, 49.544211]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [263]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
548,11765,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003959,2023-01-27,2023-04-17T09:44:53.352000+00:00
549,11766,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003958,2023-01-27,2023-04-17T09:44:53.352000+00:00
550,11767,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",51 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003955,2023-01-27,2023-04-17T09:44:53.352000+00:00
551,11768,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",50 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003954,2023-01-27,2023-04-17T09:44:53.352000+00:00


### Cohérence station - coordonnées
- 216 pdc sont associés à des stations avec plusieurs coordonnées

In [264]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1995,57921,web@freshmile.com,Freshmile,"[6.082335, 43.243236]",87 Chemin de la Pépinière 13600 La Ciotat,FRFR1PD0AIJIWGHD,"La Ciotat, 87 Chemin de la Pépinière - 3553",Voirie,4,Accès libre,24/7,false,FRFR1EKLMQ1,2022-11-09,2023-04-17T09:42:02.472000+00:00
2000,57935,web@freshmile.com,Freshmile,"[1.523894, 44.147449]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1ERHZF1,2022-11-09,2023-04-17T09:42:02.472000+00:00
2004,57944,web@freshmile.com,Freshmile,"[-0.672947, 44.859769]",87 Chemin de la Pépinière 13600 La Ciotat,FRFR1PD0AIJIWGHD,"La Ciotat, 87 Chemin de la Pépinière - 3553",Voirie,4,Accès libre,24/7,false,FRFR1EPNZZ1,2022-11-09,2023-04-17T09:42:02.472000+00:00
2007,57947,web@freshmile.com,Freshmile,"[-1.15518, 44.6395]",87 Chemin de la Pépinière 13600 La Ciotat,FRFR1PD0AIJIWGHD,"La Ciotat, 87 Chemin de la Pépinière - 3553",Voirie,4,Accès libre,24/7,false,FRFR1EPNZZ2,2022-11-09,2023-04-17T09:42:02.472000+00:00
2014,58153,web@freshmile.com,Freshmile,"[-0.749697, 46.642619]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUQWM1,2023-03-17,2023-04-17T09:42:02.472000+00:00
2015,58156,web@freshmile.com,Freshmile,"[-1.613876, 47.249316]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUSRM1,2023-03-17,2023-04-17T09:42:02.472000+00:00
2016,58167,web@freshmile.com,Freshmile,"[-1.613876, 47.249316]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUQWM2,2023-03-17,2023-04-17T09:42:02.472000+00:00
2018,58191,web@freshmile.com,Freshmile,"[-1.613876, 47.249316]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUSRM2,2023-03-17,2023-04-17T09:42:02.472000+00:00
2019,58222,web@freshmile.com,Freshmile,"[-0.749697, 46.642619]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EWKWM2,2023-03-17,2023-04-17T09:42:02.472000+00:00
2020,58223,web@freshmile.com,Freshmile,"[-0.749697, 46.642619]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EWKWM1,2023-03-17,2023-04-17T09:42:02.472000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [265]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
558,22305,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EREUO5946,2022-07-21,2023-04-17T09:44:00.082000+00:00
559,22306,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-04-17T09:44:00.082000+00:00
560,22307,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2023-04-17T09:44:00.082000+00:00
561,22308,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2023-04-17T09:44:00.082000+00:00
562,22309,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2023-04-17T09:44:00.082000+00:00
563,22310,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2023-04-17T09:44:00.082000+00:00
564,22311,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-04-17T09:44:00.082000+00:00
565,22312,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EUHNJ5886,2022-07-21,2023-04-17T09:44:00.082000+00:00
566,22313,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EYLF65474,2022-07-21,2023-04-17T09:44:00.082000+00:00
567,22314,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ETCIX5105,2022-07-21,2023-04-17T09:44:00.082000+00:00


- exemple : une station avec plusieurs coordonnées -> erreur de saisie

In [266]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FRV75PPX0704', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : deux stations avec une localisation identique mais deux adresses différentes (pb UTF8) + id identique entre station et pdc

In [267]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[0.193942, 49.544211]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Incohérence 'nbre_pdc'
- 25% des pdc ont un champ 'nbre_pdc' mal documenté

- exemple : station avec un pdc 'de regroupement' qui évite de documenter plusieurs lignes

In [268]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRLUMEACACIAS11', 
               ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
6977,FRLUMEACACIAS11,FRLUMEACACIAS11,"[3.1485441,50.7582304]",4


- exemple : station avec plusieurs pdc mais l'id_station est identique à l'id_pdc

In [269]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[2.451322, 45.66523]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
61095,FRSDGEMBDG512,FRSDGPMBDG512,"[2.451322, 45.66523]",2
61098,FRSDGEMBDG511,FRSDGPMBDG511,"[2.451322, 45.66523]",2


- exemple : station avec plusieurs pdc mais le champs a toujours une valeur de 1

In [270]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[-1.7548354193520386, 48.125067030488154]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
64586,FRIENE35353A51,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
64587,FRIENE35353A41,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
64588,FRIENE35353A31,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
64589,FRIENE35353A21,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
64590,FRIENE35353A11,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1


### cohérence station - enseigne
- exemple de station avec plusieurs noms d'enseigne

In [271]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS27PBARREOUCHEMOMORT', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
48251,48530,pascal.lhermitte@siege27.fr,SIEGE266,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTG,2021-06-15,2023-04-17T09:42:24.847000+00:00
48252,48531,pascal.lhermitte@siege27.fr,SIEGE265,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTD,2021-06-15,2023-04-17T09:42:24.847000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire

In [272]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
48377,48665,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:01,False,FRS23P230200101,2023-03-09,2023-04-17T09:42:17.112000+00:00
48378,48666,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:02,False,FRS23P230200102,2023-03-09,2023-04-17T09:42:17.112000+00:00


### cohérence coordonnées - adresse
- exemple de plusieurs stations avec des coordonnées identiques mais des adresses différentes

In [273]:
itinerance_5.loc[itinerance_5.coordonneesXY == '[2.460441, 50.78763]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1198,35782,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540011,2023-03-17,2023-04-17T09:43:11.729000+00:00
1202,36029,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540014,2023-03-17,2023-04-17T09:43:11.729000+00:00
1313,39861,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540013,2023-03-17,2023-04-17T09:43:11.729000+00:00
1314,39865,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540012,2023-03-17,2023-04-17T09:43:11.729000+00:00
1712,48747,support@passpasselectrique.fr,Pass Pass Electrique,"[2.460441, 50.78763]",1bis Rue De La Gare 59670 Bavinchove,FRH14E59054001,Bavinchove - Gare,Parking public,2,Accès libre,24/7,false,FRH14E59054001,2023-01-27,2023-04-17T09:42:05.737000+00:00
